<a href="https://colab.research.google.com/github/romenlaw/llm_playground/blob/main/pdf_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymupdf sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [21]:
from google.colab import userdata
openai_key = userdata.get('openai_key')

In [4]:
import fitz  # PyMuPDF for reading PDFs
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import openai

# Initialize embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Function to extract and chunk text from a PDF
def extract_and_chunk_pdf(pdf_path, chunk_size=500):
    doc = fitz.open(pdf_path)
    chunks = []
    for page in doc:
        text = page.get_text("text")
        for i in range(0, len(text), chunk_size):
            chunks.append(text[i:i+chunk_size])
    return chunks

def extract_and_chunk_string(text, chunk_size=500):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i+chunk_size])
    return chunks

# Function to generate embeddings for text chunks
def embed_chunks(chunks):
    embeddings = embedding_model.encode(chunks, convert_to_numpy=True)
    return np.array(embeddings)

# Function to create FAISS index for similarity search
def create_faiss_index(embeddings):
    index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance for similarity
    index.add(embeddings)  # Add embeddings to index
    return index

# Function to find relevant chunk based on query
def search_chunks(query, chunks, index, embeddings):
    query_embedding = embedding_model.encode([query], convert_to_numpy=True)
    _, indices = index.search(query_embedding, k=3)  # Retrieve top 3 relevant chunks
    return [chunks[i] for i in indices[0]]

# Function to query AI model based on retrieved text
def ask_ai_about_pdf(relevant_chunks, question, api_key):
    content = "\n".join(relevant_chunks)
    client = openai.OpenAI(api_key=api_key)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an AI that answers questions based on a provided document."},
            {"role": "user", "content": f"Document Excerpt: {content}"},
            {"role": "user", "content": f"Question: {question}"}
        ]
    )
    return response.choices[0].message.content


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
pdf_path = "./book5.pdf"  # Replace with your PDF file
chunks = extract_and_chunk_pdf(pdf_path)
embeddings = embed_chunks(chunks)
index = create_faiss_index(embeddings)

client = openai.OpenAI(api_key=api_key)

In [ ]:

query = "Did John Snow survive in the end?"
relevant_chunks = search_chunks(query, chunks, index, embeddings)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an AI that answers questions based on a provided document."},
        {"role": "user", "content": f"Document Excerpt: {relevant_chunks}"},
        {"role": "user", "content": f"Question: {query}"}
    ]
)

In [ ]:
response.choices[0].message.content

'The provided document excerpt does not explicitly state whether Jon Snow survives in the end. To answer this question, more context from the specific book or series would be needed. If this question pertains to his storyline in "A Song of Ice and Fire" or the TV series "Game of Thrones," additional information would be necessary to provide a comprehensive answer.'

In [ ]:
# Example Usage

query = "Did she get married?"
relevant_chunks = search_chunks(query, chunks, index, embeddings)

api_key = openai_key  # Replace with your OpenAI API key
answer = ask_ai_about_pdf(relevant_chunks, query, api_key)
print("Answer:", answer)

Answer: Yes, Daenerys (referred to as "Dany" in the excerpt) did get married. The text mentions her being a "woman wed" and having a "lord husband," which indicates that she is married, presumably to Hizdahr, as mentioned in the document.


In [7]:
!wget -nv -P data https://raw.githubusercontent.com/romenlaw/llm_playground/main/input.txt

2025-04-13 12:10:27 URL:https://raw.githubusercontent.com/romenlaw/llm_playground/main/input.txt [476638/476638] -> "data/input.txt" [1]


In [19]:
import nltk
from nltk.tokenize import sent_tokenize

# Download NLTK data (for sentence splitting)
nltk.download('punkt', quiet=True)

def chunk_by_characters(text, chunk_size=500):
    """Split text into chunks of ~chunk_size characters."""
    # return [{"chunk_id": i, "text": text[i:i+chunk_size].strip()}
    return [text[i:i+chunk_size].strip()
            for i in range(0, len(text), chunk_size)]

def chunk_by_sentences(text, sentences_per_chunk=5):
    """Split text into chunks of ~sentences_per_chunk sentences."""
    sentences = sent_tokenize(text)
    chunks = []
    for i in range(0, len(sentences), sentences_per_chunk):
        chunk = " ".join(sentences[i:i+sentences_per_chunk]).strip()
        if chunk:
            # chunks.append({"chunk_id": i // sentences_per_chunk, "text": chunk})
            chunks.append(chunk)
    return chunks if chunks else [{"chunk_id": 0, "text": "No sentences extracted."}]

In [20]:
with open('./data/input.txt', 'r') as file:
  text = file.read()

chunks = chunk_by_characters(text)
embeddings = embed_chunks(chunks)
index = create_faiss_index(embeddings)

# client = openai.OpenAI(api_key=api_key)

NameError: name 'api_key' is not defined

In [22]:
query = "What is the 'POS Terminal' or POSNo used for? Is it used for filtering the transactions?"
relevant_chunks = search_chunks(query, chunks, index, embeddings)

api_key = openai_key  # Replace with your OpenAI API key
answer = ask_ai_about_pdf(relevant_chunks, query, api_key)
print("Answer:", answer)

Answer: Yes, the POS Terminal or POSNo is used for filtering transactions. According to the document excerpt, the button is used to filter only credit entries for the Merchant Garnishee accounts and corresponding POS terminals in the Lookup Table (LUT).
